In [4]:
import findspark

findspark.init('/home/adilsaid/spark-3.5.1-bin-hadoop3')

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('miss').getOrCreate()

24/03/05 21:34:30 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 192.168.178.97 instead (on interface wlp1s0)
24/03/05 21:34:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/05 21:34:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
df = spark.read.csv('ContainsNull.csv', inferSchema = True, header = True)
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| NULL|
|emp2| NULL| NULL|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [1]:
# Method for dealing with missing data  Keep, Drop and Fil with other values

In [6]:
df.na.drop().show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [7]:
# Row needs atleast two non null values. to be droppped
df.na.drop(thresh=2).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| NULL|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [9]:
# Only drop if all are null values
df.na.drop(how = 'all').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| NULL|
|emp2| NULL| NULL|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [10]:
# subset param to only drop from certain columns
df.na.drop(subset=['sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [12]:
#  filling missing values
df.na.fill('FILL VALUES').show() # fill value only applied to the name since we used a str as the fill value. It wont fill in a numeric column

+----+-----------+-----+
|  Id|       Name|Sales|
+----+-----------+-----+
|emp1|       John| NULL|
|emp2|FILL VALUES| NULL|
|emp3|FILL VALUES|345.0|
|emp4|      Cindy|456.0|
+----+-----------+-----+



In [13]:
df.na.fill(0).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|  0.0|
|emp2| NULL|  0.0|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [15]:
df.na.fill('No Name', subset=['Name']).show() # Good practice to specify your subset.

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|   John| NULL|
|emp2|No Name| NULL|
|emp3|No Name|345.0|
|emp4|  Cindy|456.0|
+----+-------+-----+



In [17]:
from pyspark.sql.functions import mean

mean_val = df.select(mean(df['Sales'])).collect()
mean_val

[Row(avg(Sales)=400.5)]

In [19]:
mean_sales = mean_val[0][0]
mean_sales

400.5

In [20]:
df.na.fill(df.select(mean(df['Sales'])).collect()[0][0], subset=['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| NULL|400.5|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+

